In [8]:
import os as os

import numpy as np


from bigdatavqa.coreset import normalize_np
from bigdatavqa.datautils import DataUtils
from bigdatavqa.k3meansclustering import get_coreset_vec_and_weights
from bigdatavqa.coreset import coreset_to_graph
from bigdatavqa.k3meansclustering import get_3means_clusters_centers, get_3means_cost
from bigdatavqa.postexecution import get_k_means_accumulative_cost
import pandas as pd

In [2]:
best_cost = np.inf

qubits = 10
coreset_size = int(qubits / 2)
circuit_depth = 1
max_shots = 1000
max_iterations = 100 
number_of_experiment_runs = 5
data_location = "../data"
number_of_corsets_to_evaluate = 15
number_of_centroid_evaluation = 20

## Create coreset

In [3]:
data_utils = DataUtils(data_location)
raw_data = data_utils.load_dataset()

Data loaded from ../data/dataset.pickle


In [4]:
coreset_vectors, coreset_weights = get_coreset_vec_and_weights(
    raw_data,
    coreset_size,
    number_of_corsets_to_evaluate,
    number_of_centroid_evaluation,
)


coreset_vectors_for_graph = normalize_np(coreset_vectors, centralize=True)
coreset_weights_for_graph = normalize_np(coreset_weights, centralize=False)

coreset_graph, _ = coreset_to_graph(
    coreset_vectors_for_graph,
    coreset_weights_for_graph,
    number_of_qubits_representing_data=2,
)



In [7]:
for _ in range(number_of_experiment_runs):
    cluster_centers = get_3means_clusters_centers(
        coreset_graph,
        coreset_vectors,
        coreset_weights,
        circuit_depth,
        max_shots,
        max_iterations,
    )

    cost = get_3means_cost(raw_data, cluster_centers)
    if cost < best_cost:
        best_cost = cost
        best_cluster_centers = cluster_centers

RuntimeError: NLOpt runtime error: nlopt failure

# Analyzing results

In [ ]:
coreset_vectors = pd.DataFrame.to_numpy(coreset_df[['X', 'Y']])
weight_vectors = pd.Series.to_numpy(coreset_df['weights'])
cluster_vectors = pd.DataFrame.to_numpy(cluster_df)

non_weighted_cost = get_k_means_accumulative_cost(3, cluster_vectors, coreset_vectors)
weighted_cost = get_k_means_accumulative_cost(3, cluster_vectors, coreset_vectors, weight_vectors)

VQE_cost_value = weighted_cost